In [1]:
"""
Imports Here
"""
"""numpy and torch"""
import numpy as np
import torch

"""PIL"""
from PIL import Image

"""torchvision and utils"""
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

"""os"""
import os

/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
"""
Loading data from local file
"""

'\nLoading data from local file\n'

In [50]:
"""Assumes images have pixel values in range [0,255]"""
def getImages(trainDIRs, testDIRS):
    """Get image to tensor"""
    transform = transforms.Compose([
        transforms.PILToTensor()
    ])
    """Loading data into arrays"""
    xtrain, xtrain, xtest, ytest = [], [], [], []
    """training data"""
    size = [0, 0]
    for i, DIR in enumerate(trainDIRs):
        px = []
        j = 0
        for filename in sorted(os.listdir(DIR)):
            f = os.path.join(DIR, filename)
            img = Image.open(f)
            tensor = transform(img).float()
            tensor.require_grad = True
            px.append(tensor/255)
            j  = (j+1) % 20
            if j == 0:
                xtrain.append(torch.stack(px))
                px = []
                size[i] += 1
    xtrain = torch.stack(xtrain)
    ytrain = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0))
        
    """testing data"""
    size = [0, 0]
    for i, DIR in enumerate(testDIRs):
        px = []
        j = 0
        for filename in sorted(os.listdir(DIR)):
            f = os.path.join(DIR, filename)
            img = Image.open(f)
            tensor = transform(img).float()
            tensor.require_grad = True
            px.append(tensor/255)
            j = (j+1) % 20
            if j == 0:
                xtest.append(torch.stack(px))
                px = []
                size[i] += 1
    xtest = torch.stack(xtest)
    splitsize = int(xtest.shape[0]/2)
    xval, xtest = xtest.split(splitsize, dim=0)
    yval, ytest = torch.from_numpy(np.concatenate((np.ones(size[0]), np.zeros(size[1])), axis=0)).split(splitsize)
    return xtrain, ytrain, xtest, ytest, xval, yval

In [51]:
def createPatches(imgs, patchsize):
    (N, M, C, W, H) = imgs.shape
    (dsize, wsize, hsize) = patchsize
    """check for errors with sizing"""
    if (M % dsize != 0) or (W % wsize != 0) or (H % hsize != 0):
        raise Exception("patchsize is not appropriate")
    imgs = imgs.permute(0, 2, 1, 3, 4) # switch M and C
    size = (N, C, M // dsize, dsize, W // wsize, wsize, H // hsize, hsize)
    perm = (0, 2, 4, 6, 1, 3, 5, 7)
    imgs = imgs.reshape(size).permute(perm).flatten(1, 3).flatten(2, 5)
    return imgs #in format Nimgs, Npatches, patchsize

In [52]:
"""
Dataloader
"""

'\nDataloader\n'

In [53]:
class DatasetWrapper(Dataset):
    def __init__(self, X, y=None):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        else:
            return self.X[idx], self.y[idx]

In [55]:
trainDIRs = ['../../../AD_NC/train/AD/', '../../../AD_NC/train/NC']
testDIRs = ['../../../AD_NC/test/AD/', '../../../AD_NC/test/NC']
xtrain, ytrain, xtest, ytest, xval, yval = getImages(trainDIRs, testDIRs)

In [63]:
#xtrain = createPatches(xtrain, (4,16,16))
#xtest = createPatches(xtest, (4,16,16))
xval = createPatches(xval, (4,16,16))

In [57]:
def trainloader(batchsize=16):
    return DataLoader(DatasetWrapper(xtrain, ytrain), batchsize=batchsize, shuffle=True)

def testloader():
    return DataLoader(DatasetWrapper(xtest, ytest), batchsize=1, shuffle=True)

In [58]:
def trainshape():
    return xtrain.shape

def testshape():
    return xtest.shape

In [66]:
print(xtrain.shape)

torch.Size([1076, 1200, 1024])


In [65]:
print(xtest.shape)

torch.Size([225, 1200, 1024])


In [64]:
print(xval.shape)

torch.Size([225, 1200, 1024])
